Import opencl modules

In [1]:
from microsim.opencl.ramp.run import run_headless
from microsim.opencl.ramp.snapshot_convertor import SnapshotConvertor
from microsim.opencl.ramp.snapshot import Snapshot
from microsim.opencl.ramp.params import Params
from microsim.opencl.ramp.simulator import Simulator
import os

Load starting snapshot

In [2]:
opencl_dir = "./opencl"

snapshot_filepath = os.path.join(opencl_dir, "snapshots", "cache.npz")
snapshot = Snapshot.load_full_snapshot(path=snapshot_filepath)

Setup params

In [3]:
params = Params(
        retail_multiplier=1.0,
        primary_school_multiplier=1.0,
        secondary_school_multiplier=1.0,
        home_multiplier=1.0,
        work_multiplier=1.0,
        current_risk_beta=0.0165
    )

snapshot.params = params

Run OpenCL simulation for multiple repetitions

In [4]:
iterations = 100
repetitions = 5
summaries = []
num_seed_days = 5
use_gpu=False

for i in range(repetitions):
    # set the random seed of the model for each repetition, otherwise it is completely deterministic
    snapshot.seed_prngs(i)

    # seed initial infections using GAM initial cases
    data_dir = os.path.join(opencl_dir, "data")
    snapshot.seed_initial_infections(num_seed_days=num_seed_days, data_dir=data_dir)

    # Create a simulator and upload the snapshot data to the OpenCL device
    kernel_dir = os.path.join(opencl_dir, "ramp", "kernels")
    simulator = Simulator(snapshot, kernel_dir=kernel_dir, gpu=use_gpu)
    simulator.upload_all(snapshot.buffers)
    
    summary, final_state = run_headless(simulator, snapshot, iterations, quiet=True)
    summaries.append(summary)

FileNotFoundError: [Errno 2] No such file or directory: './opencl/ramp/kernelsramp_ua.cl'

Plot output summary data

In [ ]:
import matplotlib.pyplot as plt
from microsim.opencl.ramp.disease_statuses import DiseaseStatus


def plot_summary(summary):
    x = range(iterations)
    
    for disease_status in DiseaseStatus:
        plt.plot(x, summary.total_counts[disease_status.value], label=disease_status.name)
        
    plt.show()
    

In [ ]:
for summary in summaries:
    plot_summary(summaries)
